In [2]:
import numpy as np
from sklearn import preprocessing as pp
import pandas as pd

In [3]:
train_data_raw = pd.read_csv('train.csv', quotechar='"', skipinitialspace=True)
train_data_raw[['ScanCount']] = train_data_raw[['ScanCount']].astype(int)
train_data_raw[['DepartmentDescription']] = train_data_raw[['DepartmentDescription']].fillna('Unspecified')
# train_data_raw[['ScanCount']] = pd.to_numeric(train_data_raw[['ScanCount']], errors='ignore')
train_data_raw[0:5]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [4]:
train_data_raw.columns

Index([u'TripType', u'VisitNumber', u'Weekday', u'Upc', u'ScanCount',
       u'DepartmentDescription', u'FinelineNumber'],
      dtype='object')

In [5]:
def label_transform(col):
    le = pp.LabelEncoder()
    le.fit(col)
    label_seq = list(le.classes_)
    col_tr = le.fit_transform(col)
    return (label_seq, col_tr)

def feature_encoder(col):
    (labels, col_tr) = label_transform(col)
    p = len(labels)
    n = len(col_tr)
    n_values = np.repeat(p, n)
    enc = pp.OneHotEncoder(n_values=n_values)
    col_enc = enc.fit_transform([col_tr]).toarray()
    col_enc_mx = np.split(col_enc[0], n)
    df = pd.DataFrame(col_enc_mx)
    df.columns = labels
    return df

In [6]:
dept = feature_encoder(train_data_raw['DepartmentDescription'])
dept[0:5]

C:\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:1767: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.n_values == 'auto':
C:\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:1799: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.n_values == 'auto':


,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
dept[['FINANCIAL SERVICES', 'SHOES','PERSONAL CARE','PAINT AND ACCESSORIES']][0:5]

,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,0,0,0,1


In [8]:
# train_data_raw[['ScanCount']].convert_objects(convert_numeric=True)*2
# pd.to_numeric(train_data_raw[['ScanCount']], errors='ignore')
# train_data_raw[['ScanCount']]*2
train_data_raw['ScanCount'][0:5]

0   -1
1    1
2    1
3    2
4    2
Name: ScanCount, dtype: int32

In [9]:
# train_data_raw[['ScanCount']]*dept[['FINANCIAL SERVICES', 'SHOES','PERSONAL CARE','PAINT AND ACCESSORIES']]
dept_times_count = dept.multiply(train_data_raw['ScanCount'], axis=0)
dept_times_count[['FINANCIAL SERVICES', 'SHOES','PERSONAL CARE','PAINT AND ACCESSORIES']][0:5]

,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES
0,-1,-0,-0,-0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,2
4,0,0,0,2


In [10]:
part_2_preagg = pd.concat([train_data_raw['VisitNumber'], dept_times_count], axis=1)
part_2_preagg[0:5]

,VisitNumber,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,5,-0,-0,-0,-0,-0,-0,-0,-0,-0,...,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
part_2 = part_2_preagg.groupby('VisitNumber').sum()
part_2 = part_2.reset_index()
part_2

,VisitNumber,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,12,0,0,0,0,0,0,0,0,2,...,0,0,0,1,0,0,0,0,0,0
7,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,19,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
part_1 = train_data_raw[['TripType','VisitNumber','Weekday']].drop_duplicates()
part_1 = part_1.reset_index()
part_1

,index,TripType,VisitNumber,Weekday
0,0,999,5,Friday
1,1,30,7,Friday
2,3,26,8,Friday
3,26,8,9,Friday
4,29,8,10,Friday
5,32,35,11,Friday
6,36,41,12,Friday
7,43,21,15,Friday
8,51,6,17,Friday
9,55,42,19,Friday


In [13]:
weekday = feature_encoder(part_1['Weekday'])
weekday

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
5,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0
7,1,0,0,0,0,0,0
8,1,0,0,0,0,0,0
9,1,0,0,0,0,0,0


In [14]:
part_1 = pd.concat([part_1[['TripType','VisitNumber']], weekday], axis=1)
part_1

,TripType,VisitNumber,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,999,5,1,0,0,0,0,0,0
1,30,7,1,0,0,0,0,0,0
2,26,8,1,0,0,0,0,0,0
3,8,9,1,0,0,0,0,0,0
4,8,10,1,0,0,0,0,0,0
5,35,11,1,0,0,0,0,0,0
6,41,12,1,0,0,0,0,0,0
7,21,15,1,0,0,0,0,0,0
8,6,17,1,0,0,0,0,0,0
9,42,19,1,0,0,0,0,0,0


In [15]:
len(set(part_1['VisitNumber']))==len(part_1['VisitNumber'])

True

In [16]:
train_data_tr = pd.merge(part_1, part_2, how='left', on='VisitNumber')
# train_data_tr[['TripType']] = train_data_tr[['TripType']].astype(str)
train_data_tr

,TripType,VisitNumber,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,1-HR PHOTO,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,999,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,7,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,26,8,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,8,9,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,10,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,35,11,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,41,12,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,21,15,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,6,17,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,42,19,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
